In [1]:
%pip install pandoc pylatex

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pandoc: filename=pandoc-2.4-py3-none-any.whl size=34871 sha256=ac5e072476733297e259f65dea32f204025b52c541565555d9b17f6fd7948e5b
  Stored in directory: /Users/benedikt/Library/Caches/pip/wheels/f8/4e/c3/c8e996b2f75f503d8686cd9f4eeec04b10d1fc3694363bef13
Successfully built pandoc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pandoc]
Note: you may need to restart the kernel to use updated packages.


Plan:
1. collect docs
2. flatten docs
3. pandoc
4. cleanup

In [2]:
import os
import shutil
import glob
from pathlib import Path
import pandas as pd
from pylatex import Document, Section, Subsection, Command
import  re
import pandoc
import subprocess

In [3]:
conferences=[]
for tex_file in glob.glob("./**/*.tex", recursive=True):
    conf=tex_file.split("/")[1]
    if conf not in conferences:
        conferences.append(conf)
conferences

['2025-03-icassp',
 '2025-07-sigir',
 '2025-09-clef',
 '2024-07-icml',
 '2025-11-ieeevis']

In [11]:
base = Path("./")
for conf in conferences:
    main_doc = None
    base_conf = base / conf
    for tex_file in glob.glob(f"./{conf}/*.tex", recursive=True):
        print(f"Checking {tex_file} for main document...")
        contents = ""
        with open(tex_file, "r") as f:
            contents = f.read()
        if "\\begin{document}" in contents:
            main_doc = Path(tex_file)
            break
    print(f"Processing {conf}, main doc: {main_doc}")
    if main_doc is None:
        print(f"No main document found for {conf}, skipping.")
        continue

    markdown = ""
    result = subprocess.run(
        f"pandoc {main_doc} -t gfm --wrap=none",
        shell=True,
        capture_output=True,
        text=True,
    )
    if result.returncode == 0:
        markdown = result.stdout
    else:
        print(f"Error processing {conf}: {result.stderr}")
        continue
    # remove old md files
    year = conf.split("-")[0]
    conf_name=conf.split("-")[-1]
    output_md_file = base_conf / f"notes.md"
    with open(output_md_file, "w") as f:
        f.write(f"""---
title: {conf_name.upper()} {year}
tags: conference
date: {year}
---\n""")
        markdown=re.sub(r"\n\n", "\n",markdown)
        f.write(markdown)
    posters_file=base_conf / f"posters.tex"
    if posters_file.exists():
        shutil.move(posters_file, base_conf / f"posters.md")
    posters_saved=""
    posters_file=base_conf / f"posters.md"
    if posters_file.exists():
        with open(posters_file, "r") as f:
            posters_saved = f.read()
        with open(base_conf / f"posters.md", "w") as f:
            f.write(f"""---
title: {conf_name.upper()} {year} Posters
tags:
- conference
- posters
date: {year}
---\n""")
            f.write(posters_saved)

Checking ./2025-03-icassp/report.tex for main document...
Processing 2025-03-icassp, main doc: 2025-03-icassp/report.tex
Checking ./2025-07-sigir/report.tex for main document...
Processing 2025-07-sigir, main doc: 2025-07-sigir/report.tex
Checking ./2025-09-clef/report.tex for main document...
Processing 2025-09-clef, main doc: 2025-09-clef/report.tex
Checking ./2024-07-icml/icml_report_flat.tex for main document...
Processing 2024-07-icml, main doc: 2024-07-icml/icml_report_flat.tex
Checking ./2025-11-ieeevis/report.tex for main document...
Processing 2025-11-ieeevis, main doc: 2025-11-ieeevis/report.tex
